<a href="https://colab.research.google.com/github/Prerna0025/Machine-learning/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as func
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Available device: {device}')


Available device: cuda


In [ ]:
# First, install PyTorch and torch-scatter, torch-sparse, torch-cluster, and torch-spline-conv
# These are dependencies specific to PyTorch Geometric
!pip install torch
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv

# Then install torch-geometric
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3662080 sha256=49b9ae6375b2122d9c4272690b94a6462709a51831b3ac3b80b649ce7a7ac516
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=2795426 sha256=1504d84529d96b83a7007c3989d134de4852a0d0685d745e57ddf51fc2464ea3
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
  Created wheel for to

In [5]:
import os
folder_path = '/content/drive/My Drive'

print(os.listdir(folder_path))

['tree', 'label.txt', 'source_tweets.txt']


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving source_tweets.txt to source_tweets.txt


In [ ]:
import zipfile
import os

# Replace 'folder.zip' with the name of your uploaded zip file
with zipfile.ZipFile("tree.zip", 'r') as zip_ref:
    zip_ref.extractall("tree")


In [7]:
def tweet_source():
  tweets = {}
  with open('/content/drive/My Drive/source_tweets.txt') as l:
    for line in l:
      id,tweet = line.strip().split('\t')
      tweets[id] = tweet
  print(list(tweets.items())[:5])

tweet_source()


[('731166399389962242', '🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL'), ('714598641827246081', 'an open letter to trump voters from his top strategist-turned-defector URL via @xojanedotcom'), ('691809004356501505', '"america is a nation of second chances" —@potus on new reforms to solitary confinement: URL URL'), ('693204708933160960', 'brandon marshall visits and offers advice, support to brother of fallen hero zaevion dobson: URL URL'), ('551099691702956032', 'rip elly may clampett: so sad to learn #beverlyhillbillies star donna douglas has passed away. URL')]


In [ ]:
class twitter_Dataset:
  def __init__(self,interactions,tweets,labels):
    self.interactions = interactions
    self.tweets = tweets
    self.labels = labels

  def graph_making(self):
    nodes = {}
    edges = []
    labels = []



    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model_bert = BertModel.from_pretrained("bert-base-uncased").to(device)
